<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman; color: skyblue">
    TIME SERIES DATA PROCESSING
    <br>
    NET TRANSFER CAPACITY
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    Main Formatting Notebook
</div>
</div>
<br>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Each part of the following script was used to process the raw data for the Net Transfer Capacity Time Series Raw Data for all the european countries of the Dispa-SET_Unleash project.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    1. Notebook Set Up
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Importing needed libraries
</div>

In [1]:
import pandas as pd
from entsoe import EntsoePandasClient
from entsoe.exceptions import NoMatchingDataError  # Ensure this line is included
import os
import csv
from datetime import datetime
import pytz
from pytz import timezone, utc
import shutil

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    2. Dispa-SET_Unleash Folder Path
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [2]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    3. Usefull Variable Definition
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Entering a value to all the variables which content are going to be used in some of the next stages of this script. 
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Indicate the year of all data is referring to in the variable data_year.
</div>

In [3]:
# Additional string to be appended
additional_path = "/RawData/NTC/"
additional_path_1 = "/RawData/NTC/Raw_Data_Sources/"

# Construct the Outage_Factors_folder_path variable
ntc_folder_path = dispaSET_unleash_folder_path + additional_path

# Construct the Outage_Factors_Raw_Data_folder_path variable
ntc_raw_data_folder_path = dispaSET_unleash_folder_path + additional_path_1

print("ntc_folder_path:", ntc_folder_path)
print("ntc_raw_data_folder_path:", ntc_raw_data_folder_path)

ntc_folder_path: /home/ray/Dispa-SET_Unleash/RawData/NTC/
ntc_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/


In [4]:
# Year to which data refers to:
data_year = 2023
data_year_str = str(data_year)

# Define a list of country codes
ntc_per_unit_country_list = ["AT", "BE", "BG", "CH", "CY", "CZ", "DE", "DK", "EE", "GR", "ES", "FI", "FR", "HR", "HU", 
                             "IE", "IT", "LT", "LU", "LV", "MT", "NL", "NO", "PL", "PT", "RO", "SE", "SI", "SK", "UK"]

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
    3.1. Back Up Directory
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Saving the original files into a Back up folder.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Since in the next steps of the processing data new subfolders and files are going to be created, the original ones are saved in a back up foldet to return them as its default content ones the process will be finished.
</div>

In [5]:
additional_path_5 = '/RawData/NTC_backup/'

# Construct the backup_folder_path variable
backup_folder_path = dispaSET_unleash_folder_path + additional_path_5

print("backup_folder_path:", backup_folder_path)

# Create a backup of the directory
if os.path.exists(backup_folder_path):
    shutil.rmtree(backup_folder_path)  # Remove any existing backup if necessary
shutil.copytree(ntc_folder_path, backup_folder_path)

print(f"Backup created at {backup_folder_path}")

backup_folder_path: /home/ray/Dispa-SET_Unleash/RawData/NTC_backup/
Backup created at /home/ray/Dispa-SET_Unleash/RawData/NTC_backup/


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    4. Reference File Definition
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Creating a file where needed information to process the raw data is used in the next stages. 
</div>

In [6]:
# Define the directory and file path
file_name = 'country_list.csv'
file_path = os.path.join(ntc_folder_path, file_name)

# Ensure the directory exists
os.makedirs(ntc_folder_path, exist_ok=True)

# Create a DataFrame
df = pd.DataFrame(ntc_per_unit_country_list, columns=['Country_From'])

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

print(f"DataFrame saved to '{file_path}'")
ntc_country_list_file = file_path

DataFrame saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/country_list.csv'


<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [7]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"data_year:                                                 {data_year}")
print (f"ntc_folder_path:                                           {ntc_folder_path}")   
print (f"ntc_raw_data_folder_path:                                  {ntc_raw_data_folder_path}")
print (f"ntc_country_list_file:                                     {ntc_country_list_file}")

dispaSET_unleash_folder_name:                              Dispa-SET_Unleash
dispaSET_unleash_folder_path:                              /home/ray/Dispa-SET_Unleash
data_year:                                                 2023
ntc_folder_path:                                           /home/ray/Dispa-SET_Unleash/RawData/NTC/
ntc_raw_data_folder_path:                                  /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/
ntc_country_list_file:                                     /home/ray/Dispa-SET_Unleash/RawData/NTC/country_list.csv


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Extracting the raw data sources and upgrading its path to the ntc_country_list_file
</div>

In [8]:
# Step 1: Copy the folder and its contents
source_folder = os.path.join(ntc_raw_data_folder_path, data_year_str)
destination_folder = os.path.join(ntc_folder_path, data_year_str)

if os.path.exists(source_folder):
    shutil.copytree(source_folder, destination_folder)
    print(f"Copied {source_folder} to {destination_folder}")
else:
    print(f"Source folder {source_folder} does not exist.")

# Step 2: Update the CSV file with the paths to the subfolders
# Read the CSV file into a DataFrame
df = pd.read_csv(ntc_country_list_file)

# Initialize the new column in the DataFrame
df['Country_Folder'] = ''

# Iterate over each country in ntc_per_unit_country_list and update the DataFrame
for index, row in df.iterrows():
    country = row['Country_From']
    if country in ntc_per_unit_country_list:
        # Generate the subfolder path
        subfolder_path = os.path.join(destination_folder, country)
        
        # Update the 'Country_Folder' column for the corresponding 'Country_From'
        if os.path.exists(subfolder_path):
            df.at[index, 'Country_Folder'] = subfolder_path
            print(f"Processed folder path for country: {country}")
        else:
            print(f"Folder does not exist for country: {country}")
    else:
        print(f"Country {country} is not in the predefined list.")

# Save the updated DataFrame back to the CSV file
df.to_csv(ntc_country_list_file, index=False)
print(f"Updated {ntc_country_list_file} with the new subfolder paths.")
print("Operation completed successfully.")


Copied /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023 to /home/ray/Dispa-SET_Unleash/RawData/NTC/2023
Processed folder path for country: AT
Processed folder path for country: BE
Processed folder path for country: BG
Processed folder path for country: CH
Processed folder path for country: CY
Processed folder path for country: CZ
Processed folder path for country: DE
Processed folder path for country: DK
Processed folder path for country: EE
Processed folder path for country: GR
Processed folder path for country: ES
Processed folder path for country: FI
Processed folder path for country: FR
Processed folder path for country: HR
Processed folder path for country: HU
Processed folder path for country: IE
Processed folder path for country: IT
Processed folder path for country: LT
Processed folder path for country: LU
Processed folder path for country: LV
Processed folder path for country: MT
Processed folder path for country: NL
Processed folder path for country: NO
Processed

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Identifying the country neighbors and writing them to the ntc_country_list_file
</div>

In [9]:
# Base path where the data_year_str folder is located
base_folder_path = os.path.join(os.path.dirname(ntc_country_list_file), data_year_str)

# Read the CSV file into a DataFrame
df = pd.read_csv(ntc_country_list_file)

# Initialize columns for neighbors
max_neighbors = 0

# Iterate over each country in the ntc_per_unit_country_list
for country in ntc_per_unit_country_list:
    # Path to the subfolder for this country
    subfolder_path = os.path.join(base_folder_path, country)
    
    # Check if the subfolder exists
    if os.path.exists(subfolder_path):
        # Get the list of files in the subfolder, removing the '.csv' extension
        files = [os.path.splitext(f)[0] for f in os.listdir(subfolder_path) if f.endswith('.csv')]
        
        # Sort the files to ensure the correct order
        files.sort()
        
        # Determine the maximum number of neighbors (for column creation)
        if len(files) > max_neighbors:
            max_neighbors = len(files)
        
        # Add the filenames as Neighbor_1, Neighbor_2, etc. in the DataFrame
        for i, file_name in enumerate(files):
            neighbor_column = f'Neighbor_{i+1}'
            if neighbor_column not in df.columns:
                df[neighbor_column] = ''
            df.loc[df['Country_From'] == country, neighbor_column] = file_name
        
        print(f"Processed subfolder for country: {country}")
    else:
        print(f"Subfolder does not exist for country: {country}")

# Handle the case where some countries might have fewer neighbors
for i in range(max_neighbors):
    neighbor_column = f'Neighbor_{i+1}'
    if neighbor_column not in df.columns:
        df[neighbor_column] = ''

# Save the updated DataFrame back to the CSV file
df.to_csv(ntc_country_list_file, index=False)
print(f"Updated {ntc_country_list_file} with the neighbor filenames.")

Processed subfolder for country: AT
Processed subfolder for country: BE
Processed subfolder for country: BG
Processed subfolder for country: CH
Processed subfolder for country: CY
Processed subfolder for country: CZ
Processed subfolder for country: DE
Processed subfolder for country: DK
Processed subfolder for country: EE
Processed subfolder for country: GR
Processed subfolder for country: ES
Processed subfolder for country: FI
Processed subfolder for country: FR
Processed subfolder for country: HR
Processed subfolder for country: HU
Processed subfolder for country: IE
Processed subfolder for country: IT
Processed subfolder for country: LT
Processed subfolder for country: LU
Processed subfolder for country: LV
Processed subfolder for country: MT
Processed subfolder for country: NL
Processed subfolder for country: NO
Processed subfolder for country: PL
Processed subfolder for country: PT
Processed subfolder for country: RO
Processed subfolder for country: SE
Processed subfolder for coun

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Updating the Acronym of the country Greece in the ntc_country_list_file
</div>

In [10]:
# Read the CSV file into a DataFrame
df = pd.read_csv(ntc_country_list_file)

# Replace 'GR' with 'EL' in the entire DataFrame
df = df.applymap(lambda x: x.replace('GR', 'EL') if isinstance(x, str) else x)

# Save the updated DataFrame back to the CSV file
df.to_csv(ntc_country_list_file, index=False)

print(f"Replacements made and file saved: {ntc_country_list_file}")

Replacements made and file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/country_list.csv


/tmp/ipykernel_1411548/2584139094.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('GR', 'EL') if isinstance(x, str) else x)


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Updating the Acronym of the country Greece inside the ntc_folder_path
</div>

In [11]:
# Walk through the directory tree
for root, dirs, files in os.walk(ntc_folder_path, topdown=False):
    # Rename files
    for name in files:
        if 'GR' in name:
            new_name = name.replace('GR', 'EL')
            old_file_path = os.path.join(root, name)
            new_file_path = os.path.join(root, new_name)
            os.rename(old_file_path, new_file_path)
            print(f"Renamed file: {old_file_path} to {new_file_path}")

    # Rename directories
    for name in dirs:
        if 'GR' in name:
            new_name = name.replace('GR', 'EL')
            old_dir_path = os.path.join(root, name)
            new_dir_path = os.path.join(root, new_name)
            os.rename(old_dir_path, new_dir_path)
            print(f"Renamed directory: {old_dir_path} to {new_dir_path}")

Renamed file: /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/BG/GR.csv to /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/BG/EL.csv
Renamed file: /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/IT/GR.csv to /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/IT/EL.csv
Renamed directory: /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/GR to /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/EL
Renamed file: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/BG/GR.csv to /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/BG/EL.csv
Renamed file: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/IT/GR.csv to /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/IT/EL.csv
Renamed directory: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/GR to /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/EL


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Joining all neighbors countries files in one file.
<br>
This is done for each country.
</div>

In [12]:
# Read the CSV file into a DataFrame
df = pd.read_csv(ntc_country_list_file)

# Ensure the column 'Country_Folder' exists
if 'Country_Folder' not in df.columns:
    raise ValueError("Column 'Country_Folder' does not exist in the CSV file")

# Function to join CSV files in a directory
def join_csv_files_in_directory(directory_path):
    csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    if not csv_files:
        return None
    
    # Read all CSV files into DataFrames
    dataframes = {csv_file: pd.read_csv(os.path.join(directory_path, csv_file)) for csv_file in csv_files}
    
    # Find the CSV file with the largest number of rows
    largest_file = max(dataframes, key=lambda x: len(dataframes[x]))
    base_df = dataframes[largest_file].iloc[:, :2].copy()
    base_df.columns = [base_df.columns[0], largest_file.replace('.csv', '')]
    
    # Merge the other CSV files based on the first column
    for csv_file, df in dataframes.items():
        if csv_file == largest_file:
            continue
        temp_df = df.iloc[:, [0, 1]]
        temp_df.columns = [temp_df.columns[0], csv_file.replace('.csv', '')]
        base_df = pd.merge(base_df, temp_df, on=base_df.columns[0], how='left')
    
    return base_df

# Create a new column for the paths of the new CSV files
df['Country_File_Path'] = ''

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    country_folder = row['Country_Folder']
    
    # Join CSV files in the directory
    joined_df = join_csv_files_in_directory(country_folder)
    
    if joined_df is not None:
        # Define the output file path
        output_file = os.path.join(country_folder, f"{os.path.basename(country_folder)}.csv")
        
        # Save the joined DataFrame to a new CSV file
        joined_df.to_csv(output_file, index=False)
        
        # Update the DataFrame with the path of the new CSV file
        df.at[index, 'Country_File_Path'] = output_file

        print(f"Joined CSV file saved to '{output_file}'")

# Save the updated DataFrame back to the main CSV file
df.to_csv(ntc_country_list_file, index=False)

print("All data has been processed and saved.")

Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/AT/AT.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/BE/BE.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/BG/BG.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/CH/CH.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/CZ/CZ.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/DE/DE.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/DK/DK.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/EE/EE.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/EL/EL.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/ES/ES.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/FI/FI.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/FR/FR.csv'
Joined CSV file 

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Giving the header columns to the just joined files.
<br>
This headers indicates the direction of the power flow for each country.
</div>

In [13]:
# Read the main CSV file into a DataFrame
df = pd.read_csv(ntc_country_list_file)

# Ensure the required columns exist
if 'Country_From' not in df.columns or 'Country_File_Path' not in df.columns:
    raise ValueError("The CSV file must contain 'Country_From' and 'Country_File_Path' columns.")

# Function to update the headers of a CSV file
def update_csv_headers(file_path, new_header_prefix):
    # Read the CSV file into a DataFrame
    csv_df = pd.read_csv(file_path)
    
    # Get the current headers
    current_headers = csv_df.columns.tolist()
    
    # Create new headers for columns from the second column onward
    new_headers = [current_headers[0]] + [f"{new_header_prefix} -> {col}" for col in current_headers[1:]]
    
    # Update the DataFrame with the new headers
    csv_df.columns = new_headers
    
    # Save the updated DataFrame back to the CSV file
    csv_df.to_csv(file_path, index=False)
    print(f"Updated headers in '{file_path}'")

# Iterate through each row in the main DataFrame
for index, row in df.iterrows():
    country_from = row['Country_From']
    country_file_path = row['Country_File_Path']
    
    # Check if the file path is not empty and exists
    if pd.notna(country_file_path) and os.path.exists(country_file_path):
        update_csv_headers(country_file_path, country_from)
    else:
        print(f"File path '{country_file_path}' does not exist or is empty. Skipping...")

print("All CSV files have been processed.")

Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/AT/AT.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/BE/BE.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/BG/BG.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/CH/CH.csv'
File path 'nan' does not exist or is empty. Skipping...
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/CZ/CZ.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/DE/DE.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/DK/DK.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/EE/EE.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/EL/EL.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/ES/ES.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/FI/FI.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/NTC/2023/FR/FR.csv'
Updated headers in '/home/ray/Di

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    5. Raw Data Format
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Addapting the time step data to the UTC for all the countries.
</div>

In [14]:
# Read the country list CSV file
country_list_df = pd.read_csv(ntc_country_list_file)

# Ensure the 'Country_File_Path' column exists
if 'Country_File_Path' not in country_list_df.columns:
    raise ValueError("Column 'Country_File_Path' does not exist in the CSV file")

# Define the function to convert time to UTC
def convert_to_utc(time_str):
    local_time = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S%z')
    utc_time = local_time.astimezone(pytz.utc)
    return utc_time.strftime('%Y-%m-%d %H:%M:%S%z')

# Process each CSV file
for file_path in country_list_df['Country_File_Path'].dropna():
    # Ensure the file exists
    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        continue

    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Check if the 'index' column exists
    if 'index' not in df.columns:
        print(f"'index' column not found in file: {file_path}")
        continue

    # Convert the 'index' column to UTC
    df['index'] = df['index'].apply(convert_to_utc)
    
    # Save the updated CSV file
    df.to_csv(file_path, index=False)
    print(f"Updated file saved: {file_path}")

Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/AT/AT.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/BE/BE.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/BG/BG.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/CH/CH.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/CZ/CZ.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/DE/DE.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/DK/DK.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/EE/EE.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/EL/EL.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/ES/ES.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/FI/FI.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/FR/FR.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/HR/HR.csv
Updated file saved: /home

In [15]:
# Read the country list CSV file
country_list_df = pd.read_csv(ntc_country_list_file)

# Ensure the 'Country_File_Path' column exists
if 'Country_File_Path' not in country_list_df.columns:
    raise ValueError("Column 'Country_File_Path' does not exist in the CSV file")

# Function to update the year in the 'index' column
def update_index_year(df, data_year):
    # Ensure the 'index' column exists
    if 'index' not in df.columns:
        raise ValueError("'index' column not found in DataFrame")
    
    # Update the year in the 'index' column
    df['index'] = df['index'].apply(lambda x: f"{data_year}{x[4:]}" if str(x)[:4] != str(data_year) else x)
    
    return df

# Process each CSV file specified in the 'Country_File_Path' column
for file_path in country_list_df['Country_File_Path'].dropna():
    # Ensure the file exists
    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        continue
    
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Ensure there are enough rows to move the first four rows to the last
    if len(df) < 4:
        print(f"Not enough rows to process in file: {file_path}")
        continue
    
    # Extract the first four rows (excluding headers)
    first_four_rows = df.iloc[:4].copy()
    
    # Drop the first four rows from the DataFrame
    df = df.iloc[4:].reset_index(drop=True)
    
    # Append the first_four_rows to the end of the DataFrame
    df = pd.concat([df, first_four_rows]).reset_index(drop=True)
    
    # Update the 'index' column year
    df = update_index_year(df, data_year)
    
    # Save the updated DataFrame back to the CSV file
    df.to_csv(file_path, index=False)
    print(f"Updated file saved: {file_path}")

Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/AT/AT.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/BE/BE.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/BG/BG.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/CH/CH.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/CZ/CZ.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/DE/DE.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/DK/DK.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/EE/EE.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/EL/EL.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/ES/ES.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/FI/FI.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/FR/FR.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023/HR/HR.csv
Updated file saved: /home

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    6. Net Transfer Capacities Clean File
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Joining all the net transfer capacity data to a single csv file with named as the analized year.
</div>

In [16]:
# Read the country list CSV file
country_list_df = pd.read_csv(ntc_country_list_file)

# Ensure the 'Country_File_Path' column exists
if 'Country_File_Path' not in country_list_df.columns:
    raise ValueError("Column 'Country_File_Path' does not exist in the CSV file")

# Process each CSV file specified in the 'Country_File_Path' column
file_paths = country_list_df['Country_File_Path'].dropna().tolist()

# Identify the CSV file with the largest number of rows
max_rows = 0
base_df = None
for file_path in file_paths:
    # Ensure the file exists
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        if len(df) > max_rows:
            max_rows = len(df)
            base_df = df.copy()

# If no base_df was found, raise an error
if base_df is None:
    raise ValueError("No valid CSV files found.")

# Initialize the combined DataFrame with the first column from the base DataFrame
combined_df = pd.DataFrame(base_df.iloc[:, 0])
combined_df.columns = [base_df.columns[0]]  # Keep the original name of the first column

# Add data from each CSV file to the combined DataFrame
for file_path in file_paths:
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        # Merge the data based on the first column
        combined_df = pd.merge(combined_df, df, on=base_df.columns[0], how='left')

# Save the combined DataFrame to a new CSV file named after the data_year variable
output_file_path = os.path.join(ntc_folder_path, f"{data_year}.csv")
combined_df.to_csv(output_file_path, index=False)
print(f"Combined CSV file saved: {output_file_path}")

Combined CSV file saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/2023.csv


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Dividing the clean data in time stepts of 15 minutes, 30 minutes, and 1 hour.
</div>

In [17]:
csv_file_path = os.path.join(ntc_folder_path, f'{data_year}.csv')

# Create the new directories
intervals = ['1h', '30min', '15min']
for interval in intervals:
    os.makedirs(os.path.join(ntc_folder_path, interval), exist_ok=True)

# Read the original CSV file
df = pd.read_csv(csv_file_path)

# Convert the 'index' column to datetime
df['index'] = pd.to_datetime(df['index'], format='%Y-%m-%d %H:%M:%S%z')

# Function to extract rows at a specific time step and save to a new CSV file
def extract_and_save(df, interval, folder_name):
    # Resample the DataFrame
    resampled_df = df.set_index('index').resample(interval).first().reset_index()
    
    # Define the new file path
    new_file_path = os.path.join(ntc_folder_path, folder_name, f'{data_year}.csv')
    
    # Save the resampled DataFrame to the new CSV file
    resampled_df.to_csv(new_file_path, index=False)
    print(f"File saved: {new_file_path}")

# Extract and save rows at different time steps
extract_and_save(df, '1H', '1h')
extract_and_save(df, '30T', '30min')
extract_and_save(df, '15T', '15min')

/tmp/ipykernel_1411548/4274416861.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  resampled_df = df.set_index('index').resample(interval).first().reset_index()


File saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/1h/2023.csv


/tmp/ipykernel_1411548/4274416861.py:17: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = df.set_index('index').resample(interval).first().reset_index()


File saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/30min/2023.csv


/tmp/ipykernel_1411548/4274416861.py:17: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = df.set_index('index').resample(interval).first().reset_index()


File saved: /home/ray/Dispa-SET_Unleash/RawData/NTC/15min/2023.csv


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Correcting those non numeric or empty fields from the processed data.
</div>

In [18]:
# List of subfolders to check
subfolders = ['1h', '15min', '30min']

# Iterate through each subfolder
for subfolder in subfolders:
    # Construct the full path to the CSV file
    csv_file_path = os.path.join(ntc_folder_path, subfolder, f'{data_year}.csv')
    
    # Check if the CSV file exists
    if os.path.exists(csv_file_path):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file_path)
        
        # Process the relevant columns (from the second column onwards)
        for col in df.columns[1:]:
            # Replace non-numeric values with '0'
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        
        # Save the modified DataFrame back to the CSV file
        df.to_csv(csv_file_path, index=False)
        
        print(f"Processed and updated {csv_file_path}")
    else:
        print(f"CSV file {csv_file_path} does not exist.")

Processed and updated /home/ray/Dispa-SET_Unleash/RawData/NTC/1h/2023.csv
Processed and updated /home/ray/Dispa-SET_Unleash/RawData/NTC/15min/2023.csv
Processed and updated /home/ray/Dispa-SET_Unleash/RawData/NTC/30min/2023.csv


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Copying the time already formated Net Transfer Capacities data to the main Dispa-SET data base dirtectory
</div>

In [19]:
additional_path_2 = "/Database/NTC/"

# Construct the power_plants_raw_data_folder_path variable
ntc_data_base_folder_path = dispaSET_unleash_folder_path + additional_path_2

In [20]:
# Define the subfolder names
subfolders = ['1h', '30min', '15min']

# Function to copy files
def copy_files(data_year, source_base_path, dest_base_path, subfolders):
    for subfolder in subfolders:
        source_path = os.path.join(source_base_path, subfolder, f"{data_year}.csv")
        dest_folder_path = os.path.join(dest_base_path, subfolder)

        # Create the destination subfolder if it does not exist
        os.makedirs(dest_folder_path, exist_ok=True)

        dest_path = os.path.join(dest_folder_path, f"{data_year}.csv")
        
        # Copy the file
        if os.path.isfile(source_path):
            shutil.copy2(source_path, dest_path)
            print(f"Copied {source_path} to {dest_path}")
        else:
            print(f"File {source_path} does not exist")

# Call the function
copy_files(data_year, ntc_folder_path, ntc_data_base_folder_path, subfolders)

Copied /home/ray/Dispa-SET_Unleash/RawData/NTC/1h/2023.csv to /home/ray/Dispa-SET_Unleash/Database/NTC/1h/2023.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/NTC/30min/2023.csv to /home/ray/Dispa-SET_Unleash/Database/NTC/30min/2023.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/NTC/15min/2023.csv to /home/ray/Dispa-SET_Unleash/Database/NTC/15min/2023.csv


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
7. Net Transfer Capacity Folder Back Up
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Once all the formating process was done the Net Transfer Capacity Folder is restored to its defoult state.
</div>

In [21]:
if os.path.exists(ntc_folder_path):
    shutil.rmtree(ntc_folder_path)  # Remove the current directory
shutil.copytree(backup_folder_path, ntc_folder_path)

print(f"Directory restored to original state from {backup_folder_path}")

Directory restored to original state from /home/ray/Dispa-SET_Unleash/RawData/NTC_backup/


In [22]:
shutil.rmtree(backup_folder_path)
print(f"Backup folder {backup_folder_path} deleted successfully.")

Backup folder /home/ray/Dispa-SET_Unleash/RawData/NTC_backup/ deleted successfully.
